In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
# ad = pd.read_csv('../adFeature.csv')
# user = pd.read_csv('userFeature.csv')
train = pd.read_csv('../../tencent_ad/CJY/r2/input/train.csv')
# test = pd.read_csv('test2.csv')

# ad.shape
# ad.head()

# user.shape
# user.head()

train.shape
train.head()

# test.shape
# test.head()

(45539700, 3)

,aid,uid,label
0,627,17557009,-1
1,1502,17401944,1
2,1993,55569517,-1
3,1505,17799681,-1
4,210,10887549,-1


In [4]:
ad.groupby('productType').aid.count()

productType
4     55
6     33
9     41
11    44
Name: aid, dtype: int64

In [6]:
ad = ad.sort_values(by='aid')

ad.head()

,aid,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
142,6,7565,766460,1702986,35,94,0,4
120,7,3387,163957,221432,42,137,3826,11
149,12,388,420987,1612095,35,67,113,9
137,18,8203,74452,857791,95,218,0,6
152,70,327,5616,5977,22,27,113,9


In [7]:
ad['count'] = list(train.groupby('aid').aid.count())
ad['pos_percent'] = list(train[train.label == 1].groupby('aid').aid.count() / train.groupby('aid').aid.count())

In [8]:
productType_count = ad.groupby(
    ['productType'], as_index=False)['count'].agg({
        'count_sum':
        'sum'
    })
productType_count.head()

,productType,count_sum
0,4,3425170
1,6,1178018
2,9,1645649
3,11,2549977


In [9]:
productType_pos_percent = ad.groupby(
    ['productType'], as_index=False)['pos_percent'].agg({
        'pos_percent_mean':
        'mean'
    })
productType_pos_percent.head()

,productType,pos_percent_mean
0,4,0.047683
1,6,0.047451
2,9,0.048014
3,11,0.047945


In [4]:
split_col = ['marriageStatus', 'interest1', 'interest2', 'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3',
            'topic1', 'topic2', 'topic3','appIdInstall', 'appIdAction', 'ct', 'os']

In [5]:
def spl_col(x):
    if (type(x) == float) | (type(x) == int):
        return x
    elif type(x) == list:
        return x[0].split(';')
    else:
        return x.split(';')

# for i in tqdm(split_col):
#     user[i] = user[i].apply(spl_col)

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.model_selection import StratifiedKFold

/home/user/cuser/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# 生切
percent = 0.8

his = pd.DataFrame()
tr = pd.DataFrame()

for i in ad.aid.tolist():
    tmp = train[train.aid == i]
    n = tmp.shape[0]
    his = pd.concat([his, tmp.iloc[:int(n * percent), :]])
    tr = pd.concat([tr, tmp.iloc[int(n * percent):, :]])
    
his.shape[0] + tr.shape[0]
his.shape[0] / tr.shape[0]

In [ ]:
# kflod
kf = KFold(train.shape[0], n_folds=5, random_state=201803)

his = pd.DataFrame()
tr = pd.DataFrame()

for i, (his_idx, tr_idx) in enumerate(kf):
    his = train.iloc[his_idx]
    tr = train.iloc[tr_idx]
    break
        
his.shape[0] + tr.shape[0]
his.shape[0] / tr.shape[0]

In [4]:
train.groupby(['aid']).uid.count()

aid
1        19045
2        12151
3        10360
10      138545
11       14549
14       13013
15       90575
23        7086
25      359586
29       53282
30       10350
32       11013
33       10715
34       15171
37       20603
38        7556
43       25515
49       75283
50       20674
52       12746
53        7914
55      159646
56       60387
58      119767
59       20867
60      113260
63       21495
65        7831
66       13954
69       10111
         ...  
2159    104150
2161     46673
2165      8209
2166     14025
2167      9372
2171     24537
2174     14390
2175     93636
2177     45781
2178     15613
2179    332213
2180     24488
2181     66952
2185     32288
2186      8097
2188     13185
2190    112878
2192     79595
2194     51647
2195    512153
2202     23541
2203      6782
2204    157854
2206     27944
2207     19406
2211     20035
2212     32326
2213     30027
2215     43809
2230    135460
Name: uid, Length: 832, dtype: int64

In [4]:
# StratifiedKFold
skf = StratifiedKFold(n_splits=10, random_state=201803)

his_aid = pd.DataFrame()
tr_aid = pd.DataFrame()
train['kflod'] = 0

for i in tqdm(train.aid.unique()):
    tmp = train[train.aid == i]
    for j, (his_idx, tr_idx) in enumerate(skf.split(tmp, tmp.label)):
#         tmp_his = tmp.iloc[his_idx]
#         tmp_his['kflod'] = j
        tmp_tr = tmp.iloc[tr_idx]
        tmp_tr['kflod'] = j
#         break
#     his_aid = pd.concat([his_aid, tmp_his])
        tr_aid = pd.concat([tr_aid, tmp_tr])
    
# his_aid.shape[0] + tr_aid.shape[0]
# his_aid.shape[0] / tr_aid.shape[0]

# his_pid = pd.DataFrame()
# tr_pid = pd.DataFrame()

# for i in tqdm(ad.productId):
#     tmp = train[train.aid == i]
#     for his_idx, tr_idx in skf.split(tmp, tmp.label):
#         tmp_his = tmp.iloc[his_idx]
#         tmp_tr = tmp.iloc[tr_idx]
#         break
#     his_pid = pd.concat([his_pid, tmp_his])
#     tr_pid = pd.concat([tr_pid, tmp_tr])
    
# his_pid.shape[0] + tr_pid.shape[0]
# his_pid.shape[0] / tr_pid.shape[0]

100%|██████████| 832/832 [3:57:02<00:00, 17.09s/it]


In [5]:
tr_aid.shape
tr_aid.groupby(['kflod']).uid.count()

(45539700, 4)

kflod
0    4554723
1    4554561
2    4554393
3    4554196
4    4554032
5    4553879
6    4553719
7    4553563
8    4553408
9    4553226
Name: uid, dtype: int64

In [6]:
tr_aid[['aid', 'uid', 'kflod']].to_csv('kflod5_auid_r2(10).csv', index=None)

In [10]:
# his.shape
# his.head()

his = his_aid
his[['aid', 'uid']].to_csv('output55/his_auid.csv', index=None)

In [17]:
ad['his_aid_pos_percent'] = list(his_aid[his_aid.label == 1].groupby('aid').aid.count() / his.groupby('aid').aid.count())
ad['aid_his_tr_minus'] = abs(ad['his_aid_pos_percent'] - ad['pos_percent'])
ad[['pos_percent', 'his_aid_pos_percent', 'aid_his_tr_minus']].sort_values(by=['aid_his_tr_minus'])[-5:]

KeyboardInterrupt: 

In [11]:
his = his_aid
his_pos = his[his.label == 1]

his = his.merge(user, on=['uid'], how='left')
his_pos = his_pos.merge(user, on=['uid'], how='left')

his.shape
his.head()
his_pos.shape
his_pos.head()

(4399324, 26)

,aid,uid,label,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house
0,177,64616129,1,1,1,10,5,0,809.0,70;12;47;76;28;106;41;6;42;26;88;9;17;35;56;11...,47;32,NaN,NaN,52;72;92;131;63;116;13;109;8;113;28;99;9;75;20...,511791;360372;53621;91921;38283,9363;32356;96549,NaN,3640;3282;5344;2020;7117,9853;3216;9474;6584;9140,NaN,NaN,NaN,3;1,1,0,1.0
1,177,64657032,1,2,1,13;10,3,1,161.0,70;109;12;76;59;49;6;119;111;88;116;36;11,73;9;74,NaN,NaN,104;72;33;92;37;116;13;1;109;34;8;30;50;28;6;4...,331572;295120;208572;405681;556674,79250;77181;117548;105115;71854,NaN,7772;9464;2636;2277;94,8835;3441;5310;4686;4182,NaN,NaN,NaN,1;3,1;2,3,1.0
2,177,23531713,1,4,2,11,6,1,392.0,93;70;37;109;12;75;33;8;29;49;6;46;36;34;11;11...,79;6;18;68,6;4;10;1;7;5;2,NaN,100;72;92;37;116;47;78;71;109;8;30;50;28;6;42;...,338851;200216;529100;125748;521474,17341;80263;12933;71121;117462,39804;43606;55758;22819;55192,8726;5067;3450;9826;6647,7996;2295;2979;2060;6362,6111;9274;4955;312;360,50700;37791;39799;42026;41049;24753;8257;55984...,NaN,3;1,1;2,1,1.0
3,177,54155229,-1,5,2,6;13,6,1,921.0,93;98;47;75;29;59;49;15;114;46;36;78;11,73;6;21;19;13;74,NaN,NaN,127;52;100;131;63;116;13;71;109;12;8;6;46;121;...,293874;750628;47858;268355;124894,80263;9276;97650;49879;74124,NaN,7668;9952;8088;1456;5666,9362;8935;5221;7087;218,NaN,NaN,NaN,1;3,2,1,NaN
4,177,18946881,-1,1,1,10,6,0,964.0,70;118;109;75;76;8;16;29;49;6;39;71;36;11,63;30,NaN,NaN,52;100;31;131;116;78;8;30;6;42;5;46;121;59;64;129,658528;431736;600337;504089;751839,104075;107661;18780;76657;117387,NaN,5382;2321;2955;6931;9826,2700;3104;3014;105;7477,NaN,NaN,NaN,1;3,1,1,1.0


(210939, 26)

,aid,uid,label,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house
0,177,64616129,1,1,1,10,5,0,809.0,70;12;47;76;28;106;41;6;42;26;88;9;17;35;56;11...,47;32,NaN,NaN,52;72;92;131;63;116;13;109;8;113;28;99;9;75;20...,511791;360372;53621;91921;38283,9363;32356;96549,NaN,3640;3282;5344;2020;7117,9853;3216;9474;6584;9140,NaN,NaN,NaN,3;1,1,0,1.0
1,177,64657032,1,2,1,13;10,3,1,161.0,70;109;12;76;59;49;6;119;111;88;116;36;11,73;9;74,NaN,NaN,104;72;33;92;37;116;13;1;109;34;8;30;50;28;6;4...,331572;295120;208572;405681;556674,79250;77181;117548;105115;71854,NaN,7772;9464;2636;2277;94,8835;3441;5310;4686;4182,NaN,NaN,NaN,1;3,1;2,3,1.0
2,177,23531713,1,4,2,11,6,1,392.0,93;70;37;109;12;75;33;8;29;49;6;46;36;34;11;11...,79;6;18;68,6;4;10;1;7;5;2,NaN,100;72;92;37;116;47;78;71;109;8;30;50;28;6;42;...,338851;200216;529100;125748;521474,17341;80263;12933;71121;117462,39804;43606;55758;22819;55192,8726;5067;3450;9826;6647,7996;2295;2979;2060;6362,6111;9274;4955;312;360,50700;37791;39799;42026;41049;24753;8257;55984...,NaN,3;1,1;2,1,1.0
3,177,72589749,1,5,2,6;13,7,1,688.0,70;86;76;48;28;16;59;49;83;27;64;6;42;19;17;58...,24;19;13;30,NaN,NaN,72;92;116;13;78;71;109;8;30;50;28;6;42;76;27;9...,459973;64588;571611;9998;407197,74010;32918;67882;116802;20957,NaN,7632;9826;4244;8402;2463,5183;1066;13;1423;4107,NaN,NaN,NaN,1;3,2,1,1.0
4,177,75005896,1,2,2,10,6,2,360.0,70;76;75;29;49;18;95;36;11,1;46;70;4;28;48;14,NaN,NaN,116;78;71;109;30;6;42;5;76;27;89;64;129;56;120...,584361;176570;596338;609819;351040,44360;74061;44005;11395;79112,NaN,4620;1394;3196;5867;7530,6586;9108;506;8291;803,NaN,NaN,NaN,3;1,1,1,NaN


In [13]:
for i in split_col:
    his[i] = his[i].apply(spl_col)
print('Finish his')

for i in split_col:
    his_pos[i] = his_pos[i].apply(spl_col)
print('Finish his_pos')

Finish his
Finish his_pos


In [14]:
his.to_csv('output55/his.csv', index=None)
his_pos.to_csv('output55/his_pos.csv', index=None)

In [14]:
from collections import defaultdict, Counter
import ast

In [13]:
def save_obj(obj, name):
    with open('output/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('output/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [16]:
marriageStatus_dict = defaultdict(lambda: 0)
interest1_dict = defaultdict(lambda: 0)
interest2_dict = defaultdict(lambda: 0)
interest3_dict = defaultdict(lambda: 0)
interest4_dict = defaultdict(lambda: 0)
interest5_dict = defaultdict(lambda: 0)
kw1_dict = defaultdict(lambda: 0)
kw2_dict = defaultdict(lambda: 0)
kw3_dict = defaultdict(lambda: 0)
topic1_dict = defaultdict(lambda: 0)
topic2_dict = defaultdict(lambda: 0)
topic3_dict = defaultdict(lambda: 0)
appIdInstall_dict = defaultdict(lambda: 0)
appIdAction_dict = defaultdict(lambda: 0)
ct_dict = defaultdict(lambda: 0)
os_dict = defaultdict(lambda: 0)

for j in range(0, his_pos.shape[0]):
    for i in split_col:
        if type(his_pos[i][j]) == list:
            for word in his_pos[i][j]:
                locals()[i + '_dict'][word] += 1
    if j == his_pos.shape[0]-1:
        for i in split_col:
            tmp_dict = ast.literal_eval(str(Counter(locals()[i + '_dict']))[8:-1])
            save_obj(tmp_dict, str(i + '_pos_dict'))

In [16]:
for i in split_col:
    locals()[i + '_dict'] = load_obj(str(i + '_pos_dict'))
    print(str(i + '_dict') + ':' + str(len(locals()[i + '_dict'])))

marriageStatus_dict:12
interest1_dict:121
interest2_dict:80
interest3_dict:10
interest4_dict:10
interest5_dict:136
kw1_dict:96644
kw2_dict:22787
kw3_dict:4158
topic1_dict:9892
topic2_dict:9435
topic3_dict:3022
appIdInstall_dict:40552
appIdAction_dict:3899
ct_dict:5
os_dict:3


In [3]:
his_label = pd.read_csv('output55/his_auid.csv')
his_label['is_his'] = 1

train = train.merge(his_label, on=['aid', 'uid'], how='left')

def is_his_label(x):
    if x == 1:
        return x
    else:
        return 0
train['is_his'] = train['is_his'].apply(is_his_label)

In [5]:
test['label'] = -99
test['is_his'] = -99

df = pd.concat([train, test])
df.shape

(11064803, 4)

In [6]:
# df = pd.read_csv('df.csv')

# df.columns = ['index', 'aid', 'uid', 'label', 'is_his']
# df.shape
# df.head()

df.to_csv('df.csv', index=None)

In [36]:
# tr = df[df.is_his != 1]
# tr.shape

# tr = tr.merge(user, on=['uid'], how='left')
# for i in tqdm(split_col):
#     tr[i] = tr[i].apply(spl_col)

for i in tqdm(split_col):
    tmp_list = list(np.zeros(tr.shape[0]))
    for k in range(0, tr.shape[0]):
        if type(tr[i][k]) == list:
            for j in tr[i][k]:
                if j in locals()[i + '_dict'].keys():
                    tmp_list[k] = tmp_list[k] + locals()[i + '_dict'][j]
    tr[str(i + 'count_score')] = tmp_list


100%|██████████| 16/16 [37:40<00:00, 141.28s/it]


In [37]:
tr.to_csv('tr_processing.csv', index=None)